In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Scraping

In [8]:
def scrape_page(page_n):
    url = f"https://www.quandoo.fi/en/helsinki?districtFilter=3637&bookable=true&onlySpecialOffers=false&page={page_n}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rest_cards = soup.find_all(attrs={"data-qa": "merchant-card"})

    rest_names = [card.find('h3').text.strip() for card in rest_cards]
    rest_locations = [card.find(attrs={"data-qa": "merchant-location"}).text.strip() for card in rest_cards]
    # "Located in ... area" part removed in Location column.
    rest_locations = [location.replace('Located at ','').replace('area','') for location in rest_locations]
    
    rest_cuisines = [card.find(attrs={"data-qa": "merchant-card-cuisine"}).text.strip() for card in rest_cards]
    #Cleaned Cuisine's column data
    rest_cuisines = [cuisine.replace(" Restaurant","") for cuisine in rest_cuisines]

    rest_meals_boxes = [card.find(attrs={"data-qa": "merchant-meal"}) for card in rest_cards]
    rest_meals = [box.text.strip() if box else None for box in rest_meals_boxes]
    
    rest_rating_boxes = [card.find(attrs={"data-qa": "reviews-score"}) for card in rest_cards]
    rest_ratings = [float(box.text.split('/')[0].strip()) if box else None for box in rest_rating_boxes]

    rest_review_boxes = [card.find(class_="sc-1atis9w-3 dfyExP") for card in rest_cards]
    rest_review_counts = [int(box.text.split()[0].strip()) if box else None for box in rest_review_boxes]

    rest_price_boxes = [card.find(class_=re.compile(r'.*price-indicator')) for card in rest_cards]
    rest_price_levels = [len(box.find_all(class_=re.compile(r'.*oGCHK'))) for box in rest_price_boxes]

    rest_page_urls = [card.find('a')['href'] for card in rest_cards]

    df = pd.DataFrame({'Name': rest_names, 
                    'Location': rest_locations,
                    'Cuisine': rest_cuisines,
                    'Meals': rest_meals,
                    'Price Level (out of 4)': rest_price_levels,
                    'Rating (out of 6)': rest_ratings,
                    'Review Count': rest_review_counts,
                    'Page URL': rest_page_urls
                    })

    return df

In [9]:
restaurant_data = pd.DataFrame()

for page_n in range(1, 15):
    page_data = scrape_page(page_n)
    restaurant_data = pd.concat([restaurant_data, page_data], ignore_index=True)

restaurant_data

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
0,Luovuus kukkii kaaoksesta,Kaartinkaupunki,International,Meals: Dinner,3,5.8,470.0,/en/place/luovuus-kukkii-kaaoksesta-90397/menu
1,Ravintola MyStuu,Punavuori,Swiss,"Meals: Lunch, Dessert, Dinner",3,5.7,293.0,/en/place/ravintola-my-stuu-98898/menu
2,Gaucho,City Centre,Brazilian,Meals: Dinner,3,5.6,1118.0,/en/place/gaucho-105125/menu
3,Finlandia Caviar,City Centre,Gourmet,"Meals: Lunch, Dinner",3,5.6,143.0,/en/place/finlandia-caviar-15896/menu
4,Restaurant Armenian House,Kamppi,International,"Meals: Lunch, Dinner",2,5.6,316.0,/en/place/armenian-house-55148/menu
...,...,...,...,...,...,...,...,...
337,Black Sea Kitchen,Kaartinkaupunki,Georgian,"Meals: Lunch, Dinner",3,NaN,NaN,/en/place/black-sea-kitchen-109077/about
338,Mad Finn Brewing Co. Taproom Helsinki,Sompasaari,Pizza,Meals: Dinner,2,6.0,2.0,/en/place/mad-finn-brewing-co-taproom-helsinki...
339,Kahvila Mutteri,Lauttasaari,Dessert,Meals: Cake & Coffee,2,NaN,NaN,/en/place/kahvila-mutteri-100773/menu
340,Merisali - Hilton Kalastajatorppa,Munkkiniemi,Scandinavian,"Meals: Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...


In [10]:
restaurant_data.to_csv("restaurant_data.csv", sep="\t")

## Prettifying

In [11]:
data = pd.read_csv("restaurant_data.csv", sep="\t", index_col=0)
data.sample(10)

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
16,Osteria dei Mancini,Punavuori,Italian,"Meals: Dessert, Dinner",3,5.4,390.0,/en/place/osteria-dei-mancini-104058/about
88,Relove Stockmann Helsinki,City Centre,European,"Meals: Breakfast, Lunch, Dessert, Brunch",2,4.9,73.0,/en/place/relove-stockmann-helsinki-95230/menu
182,Open Lobby - Ruoholahti,Ruoholahti,Scandinavian,"Meals: Breakfast, Lunch, Dessert, Dinner",3,4.3,19.0,/en/place/holiday-inn-open-lobby-ruoholahti-63...
30,Yeastie Boi,Kamppi,Eat & Drink,"Meals: Lunch, Dinner",2,5.5,58.0,/en/place/yeastie-boi-98651/menu
271,Restaurant Royal Nepal Helsinki,Vallila,Nepalese,"Meals: Lunch, Dinner",2,NaN,NaN,/en/place/restaurant-royal-nepal-helsinki-1032...
112,Ginza Sushi Bar Kluuvi,City Centre,Japanese,"Meals: Lunch, Dinner",3,4.6,92.0,/en/place/ginza-sushi-bar-kluuvi-105182/menu
68,Ravintola Central,Ullanlinna,International,"Meals: Lunch, Dinner",2,5.0,491.0,/en/place/ravintola-central-60734/menu
203,Ravintola Base Camp Helsinki,City Centre,Nepalese,"Meals: Lunch, Dessert, Dinner",3,5.5,6.0,/en/place/ravintola-base-camp-18140/about
328,Pyöreä Sali - Hilton Kalastajatorppa,Munkkiniemi,International,Meals: Buffet,3,4.8,112.0,/en/place/pyorea-sali-hilton-kalastajatorppa-8...
37,GOODWIN The Steak House,City Centre,Steak,"Meals: Lunch, Dessert, Dinner",3,5.2,432.0,/en/place/goodwin-the-steak-house-33416/menu
